In [21]:
import pandas as pd

In [34]:
df = pd.read_csv('steam_games.csv')
df = df[df['name'].notna()]
df.drop(["discount_price", "publisher", "developer", "release_date", "desc_snippet", "types", "achievements", "recent_reviews", "mature_content"], axis=1, inplace=True)

In [35]:
idx = 0
for row in df.iterrows():
    if idx % 5000 == 0:
        print(((idx + 1) / 40800) * 100, "%")
    idx += 1
    if row[1][0].find("/bundle/") > -1:
        df.drop(row[0], inplace=True)
        #print("deleting: ", row[1][1])
        continue
    if row[1][0].find("/sub/") > -1:
        df.drop(row[0], inplace=True)
        #print("deleting: ", row[1][1])

0.0024509803921568627 %
12.257352941176471 %
24.512254901960784 %
36.7671568627451 %
49.02205882352941 %
61.27696078431373 %
73.53186274509804 %
85.78676470588236 %
98.04166666666667 %


In [37]:
for row in df.iterrows():
    if pd.isna(row[1][3]) and pd.isna(row[1][6]):
        print("deleting: ", row[1][1])
        df.drop(row[0], inplace=True)

deleting:  CRYENGINE - Sample Assets
deleting:  Worms Reloaded: The "Pre-order Forts and Hats" DLC Pack
deleting:  Krater - Soundtrack
deleting:  Dustforce Soundtrack
deleting:  The Incredible Adventures of Van Helsing II - OST
deleting:  D4: SWERY's Choice Costume Set -4 Cups of Coffee-
deleting:  D4: SWERY's Choice Costume Set -4 Bottles of Tequila-
deleting:  DayZ Tools
deleting:  Killing Floor "London's Finest" Character Pack
deleting:  Dungeon Defenders Capture the Flag Pre-Alpha Pass (Free DLC)
deleting:  Oxenfree - OST
deleting:  Super Lemonade Factory Demo
deleting:  Assault Android Cactus Original Soundtrack
deleting:  Pillar Original Soundtrack
deleting:  BrainBread 2 Mod Tools
deleting:  Gone Home Soundtrack
deleting:  SKYHILL Demo
deleting:  Cosmophony OST
deleting:  Blues and Bullets - Original Soundtrack
deleting:  Train Simulator: Western Pacific GP20 High Nose Loco Add-On
deleting:  Champions of Breakfast - OST
deleting:  Codename Panzers Phase One Soundtrack
deleting: 

In [38]:
duplicated_items = df[df["name"].duplicated()]["name"].values.tolist()
games = {}
print(df.shape)
for row in df.iterrows():
    if row[1][1] in duplicated_items:
        if row[1][1] in games:
            games[row[1][1]].append(row[1].isna().sum())
        else:
            games[row[1][1]] = [row[1].isna().sum()]
for item in games:
    games[item].sort()
    count = 0
    for row in df.iterrows():
        if row[1][1] == item and (row[1].isnull().sum() == games[item][0]):
            df.drop(row[0], inplace=True)
            count += 1
            print("deleting: ", row[1][1])
        
        if count + 1 == len(games[item]):
            break
print(df.shape)
df.to_csv("steam_games_dup_removed.csv", index=False)

(37905, 11)
deleting:  RUSH
deleting:  Batman: Arkham City - Game of the Year Edition
deleting:  Angels of Death
deleting:  Luna
deleting:  Flashback
deleting:  New York Bus Simulator
deleting:  Warhammer Quest
deleting:  City Builder
deleting:  Killing Time
deleting:  Hide and Seek
deleting:  Ultimate Arena
deleting:  Black Rose
deleting:  Solstice
deleting:  Surge
deleting:  Bounce
deleting:  Chaos Theory
deleting:  Rumpus
deleting:  Space Maze
deleting:  Spark
deleting:  The Mine
deleting:  Ashes
deleting:  Dodge
deleting:  The Deer
deleting:  Zombie Apocalypse
deleting:  Afterlife
deleting:  Dishonored - Void Walker Arsenal
deleting:  Taxi
deleting:  Alter Ego
deleting:  Ramen
deleting:  Exodus
deleting:  Exodus
deleting:  Bounty Hunter
deleting:  Escape
deleting:  Mars 2030
deleting:  Castles
deleting:  Perfect
deleting:  Escape Room
deleting:  Prisoner
deleting:  The Tower
deleting:  Solitaire
deleting:  Experience
deleting:  Squares
deleting:  Nightmare
deleting:  Exception
dele

In [39]:
df = pd.read_csv('steam_games_dup_removed.csv')

all_review_cols = ["all_review_sentiment", "all_review_sentiment_num", "all_review_num"]
temp_df = []
idx = 0
for row in df.iterrows():
    if row[1][2] == None or pd.isna(row[1][2]):
        temp_df.append(["None", 0, 0])
    else:
        try:
            items = row[1][2].split('(')
            sentiment = items[0].lower().strip()[0:-1]
            items = items[1].split(')')
            total = int(items[0].strip().replace(',', ''))
            percent = int(items[1].split('%')[0].strip()[3:]) / 100
            temp_df.append([sentiment, int(total * percent), total])
        except:
            temp_df.append(["None", 0, 0])
    idx += 1
    if idx % 5000 == 0:
        print(((idx + 1) / 40752) * 100, "%")
all_review_df = pd.DataFrame(temp_df, columns=all_review_cols)
df.drop(["all_reviews"], axis=1, inplace=True)
df = pd.concat([df, all_review_df], axis=1)

df['all_review_sentiment'] = df['all_review_sentiment'].astype('category').cat.codes
df['popular_tags'].fillna(df['genre'], inplace=True)

for i in range(len(df)):
    if pd.isna(df.loc[i, 'genre']):
        df.drop(i, inplace=True)
        
df.drop(['minimum_requirements', 'recommended_requirements', 'game_description'], axis=1, inplace=True)
df.to_csv("steam_games_reviews.csv", index=False)

categories = list(df['all_review_sentiment'].astype('category').cat.categories)

12.271790341578328 %
24.541126815861798 %
36.810463290145265 %
49.07979976442874 %
61.34913623871221 %
73.61847271299568 %
85.88780918727915 %


In [50]:
def format_csv(data):
    try:
        return list(set(data.split(',')))
    except:
        return []

df = pd.read_csv('steam_games_reviews.csv')
df['total'] = df['popular_tags'] + df['genre'] + df['game_details']
df['total'] = df['total'].apply(format_csv)
df['languages'] = df['languages'].apply(format_csv)
df.drop(['popular_tags', 'game_details', 'genre'], axis=1, inplace=True)

df.to_csv('steam_games_genres.csv', index=False)

In [51]:
df = pd.read_csv('steam_games_genres.csv')
df['all_review_sentiment'] =(df['all_review_sentiment']-df['all_review_sentiment'].min())/(df['all_review_sentiment'].max()-df['all_review_sentiment'].min())
df['all_review_sentiment_num'] =(df['all_review_sentiment_num']-df['all_review_sentiment_num'].min())/(df['all_review_sentiment_num'].max()-df['all_review_sentiment_num'].min())
df['all_review_num'] =(df['all_review_num']-df['all_review_num'].min())/(df['all_review_num'].max()-df['all_review_num'].min())

df.to_csv('steam_game_review_normal.csv', index=False)

In [52]:
df = pd.read_csv('steam_game_review_normal.csv')

def format_price(price):
    price_str = str(price)
    if price_str[0] == "$":
        return float(price_str[1:])
    
    try:
        test = int(price_str[0])
        return float(price_str)
    except:
        return 0

df['original_price'] = df['original_price'].apply(format_price)

df.to_csv('steam_games_final_format.csv', index=False)

In [43]:
df

,url,name,languages,original_price,all_review_sentiment,all_review_sentiment_num,all_review_num,total
0,https://store.steampowered.com/app/379720/DOOM/,DOOM,"['Traditional Chinese', 'Portuguese - Brazil',...",19.99,1.000000,0.014711,0.013751,"['Horror', 'Partial Controller Support', 'Stea..."
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,"['Korean', 'Traditional Chinese', 'Turkish', '...",29.99,0.111111,0.154052,0.270377,"['Tactical', 'Adventure', 'Competitive', 'Mass..."
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,"['Russian', 'French', 'German', 'English']",39.99,0.333333,0.001876,0.002272,"['Turn-Based', 'Tactical', 'RPG', 'Adventure',..."
3,https://store.steampowered.com/app/221100/DayZ/,DayZ,"['Traditional Chinese', 'Simplified Chinese', ...",44.99,0.111111,0.038308,0.054009,"['Horror', 'Survival Horror', 'MilitaryAction'..."
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,"['Russian', 'French', 'German', 'English']",0.00,0.333333,0.003192,0.003710,"['Tactical', 'Online Co-op', 'RPG', 'PvE', 'Ma..."
...,...,...,...,...,...,...,...,...
37573,https://store.steampowered.com/app/899836/Rock...,Rocksmith® 2014 Edition – Remastered – Sabaton...,"['Italian', 'French', 'Spanish - Spain', 'Japa...",2.99,0.000000,0.000000,0.000000,"['SimulationSingle-player', 'Downloadable Cont..."
37574,https://store.steampowered.com/app/899832/Rock...,Rocksmith® 2014 Edition – Remastered – Stone T...,"['Italian', 'French', 'Spanish - Spain', 'Japa...",2.99,0.000000,0.000000,0.000000,"['SimulationSingle-player', 'Downloadable Cont..."
37575,https://store.steampowered.com/app/906840/Fant...,Fantasy Grounds - Quests of Doom 4: A Midnight...,['English'],7.99,0.000000,0.000000,0.000000,"['Turn-Based', 'Party-Based RPG', 'RPG', 'Cros..."
37576,https://store.steampowered.com/app/906635/Mega...,Mega Man X5 Sound Collection,"['Traditional Chinese', 'Italian', 'French', '...",9.99,0.000000,0.000000,0.000000,"['Full controller support', 'ActionActionSingl..."
